In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder,MinMaxScaler,OrdinalEncoder

from sklearn.model_selection import train_test_split


import folium
import plotly.express as px

from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
from sklearn.linear_model import LinearRegression

import time

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

import scipy as sp

import os

In [2]:
PATH_FOLDER = os.path.join(os.getcwd(), "")

In [3]:
df=pd.read_csv("feature_eng_data.csv")

In [4]:
df.set_index('Index', inplace=True)

In [5]:
df['pk_partition'] = pd.to_datetime(df['pk_partition'])

In [6]:
df

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,...,em_acount,country_id,region_code,gender,age,deceased,salary,Financiacion,Ahorro,Cuentas
Index,,,,,,,,,,,,,,,,,,,,,
0,1375586,2018-01-28,2018-01-12,KHL,1.0,2,0,0,0,0,...,1,0,29.0,0,35,0,87218.10,0,0,1
1,1050611,2018-01-28,2015-08-10,KHE,0.0,3,0,0,0,0,...,1,0,13.0,1,23,0,35548.74,0,0,1
2,1050612,2018-01-28,2015-08-10,KHE,0.0,3,0,0,0,0,...,1,0,13.0,1,23,0,122179.11,0,0,1
3,1050613,2018-01-28,2015-08-10,KHD,0.0,3,1,0,0,0,...,0,0,50.0,0,22,0,119775.54,0,1,0
4,1050614,2018-01-28,2015-08-10,KHE,1.0,3,0,0,0,0,...,1,0,50.0,1,23,0,96796.23,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5962919,1166765,2019-05-28,2016-08-14,KHE,0.0,3,0,0,0,0,...,1,0,50.0,1,22,0,43912.17,0,0,1
5962920,1166764,2019-05-28,2016-08-14,KHE,0.0,3,0,0,0,0,...,1,0,26.0,1,23,0,23334.99,0,0,1
5962921,1166763,2019-05-28,2016-08-14,KHE,1.0,2,0,0,0,0,...,1,0,50.0,0,47,0,96796.23,0,0,1


In [7]:
df['diff_productos'] = df.groupby('pk_cid')['Cuentas'].diff()

In [8]:
df['diff_productos']

Index
0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
          ... 
5962919    0.0
5962920    0.0
5962921    0.0
5962922    0.0
5962923    NaN
Name: diff_productos, Length: 5936633, dtype: float64

In [9]:
df['compra'] = (df['diff_productos'] >= 1).astype(int)

In [10]:
df['compra'].value_counts()

compra
0    5851186
1      85447
Name: count, dtype: int64

In [11]:
#Comprobamos con un pk_cid que sabemso que tiene mas de un producto y vemos que detecta las compras correctamente

df[df['pk_cid'] == 16502]

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,...,region_code,gender,age,deceased,salary,Financiacion,Ahorro,Cuentas,diff_productos,compra
Index,,,,,,,,,,,,,,,,,,,,,
2171490,16502,2018-09-28,2018-09-30,KHE,0.0,3,0,0,0,0,...,28.0,0,57,0,126765.57,0,0,0,NaN,0
2943633,16502,2018-10-28,2018-09-30,KHN,1.0,2,0,0,0,0,...,28.0,0,58,0,126765.57,0,0,1,1.0,1
2962972,16502,2018-11-28,2018-09-30,KHN,1.0,2,0,0,0,0,...,28.0,0,58,0,126765.57,0,0,1,0.0,0
3628237,16502,2018-12-28,2018-09-30,KHN,1.0,2,0,0,0,0,...,28.0,0,58,0,126765.57,0,0,1,0.0,0
4028330,16502,2019-01-28,2018-09-30,KHN,1.0,2,0,0,0,0,...,28.0,0,58,0,126765.57,0,0,2,1.0,1
4480638,16502,2019-02-28,2018-09-30,KHN,1.0,2,0,0,0,0,...,28.0,0,58,0,126765.57,0,0,2,0.0,0
4876038,16502,2019-03-28,2018-09-30,KHN,1.0,2,0,0,0,0,...,28.0,0,58,0,126765.57,0,0,2,0.0,0
5134320,16502,2019-04-28,2018-09-30,KHN,1.0,2,0,0,0,0,...,28.0,0,58,0,126765.57,0,0,2,0.0,0
5523395,16502,2019-05-28,2018-09-30,KHN,1.0,2,0,0,0,0,...,28.0,0,58,0,126765.57,0,0,2,0.0,0


In [12]:
#Llenamos los NaN del primer mes con '0' ya que no tenemos con que comparar 

In [13]:
df['diff_productos'].fillna(value=0, inplace=True)


In [14]:
#Por tanto 'compra' sera nuestro target para la predisposicion de compra de los productos "CUENTAS"

In [15]:
df['entry_date'] = pd.to_datetime(df['entry_date'])
df['pk_partition'] = pd.to_datetime(df['pk_partition'])

In [16]:
df['Antiguedad'] = (df['pk_partition'] - df['entry_date']) / pd.Timedelta(days=30)


In [17]:
df['Antiguedad'] = df['Antiguedad'].round().astype(int)

In [18]:
df['mes'] = df['pk_partition'].dt.to_period('M')


In [19]:
serie_temporal = df.groupby('mes')['compra'].sum()


In [20]:
serie_temporal

mes
2018-01       0
2018-02    3592
2018-03    3592
2018-04    3168
2018-05    3580
2018-06    3946
2018-07    3955
2018-08    6215
2018-09    5386
2018-10    7016
2018-11    6404
2018-12    6779
2019-01    5351
2019-02    7659
2019-03    6039
2019-04    6031
2019-05    6734
Freq: M, Name: compra, dtype: int32

In [21]:
serie_temporal = serie_temporal.drop(serie_temporal.index[0])

In [22]:
#Intentaremos predecir cuantos productos de CUENTAS venderemos el mes siguiente (JUNIO DE 2019)

df_regresion = serie_temporal.reset_index()
df_regresion['mes'] = df_regresion['mes'].astype(str)  # Convertimos el periodo a string para la regresión lineal

# Dividimos los datos en características (X) y la variable objetivo (y)
X = pd.to_numeric(df_regresion['mes'].str.replace('-', ''), errors='coerce').values.reshape(-1, 1)  # Convertimos el periodo a un formato numérico para la regresión lineal
y = df_regresion['compra'].values

# Entrenamos el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X, y)

# Predecimos las compras del mes 06 del año 2019
mes_06_2019 = pd.to_numeric('2019-06'.replace('-', ''), errors='coerce').reshape(-1, 1)  # Convertimos el periodo a un formato numérico para la predicción
prediccion_mes_06_2019 = modelo.predict(mes_06_2019)

print("Predicción de compras para el mes 06 del año 2019:", prediccion_mes_06_2019[0])

Predicción de compras para el mes 06 del año 2019: 6504.497877358459


In [23]:
#Predicción de beneficios

Beneficios = prediccion_mes_06_2019[0]*10
Beneficios

#Beneficios estimados para JUNIO 2019 de cproductos de CUENTAS = 65.044 € 

65044.97877358459

### Predicción de acierto del modelo

In [24]:
df

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,...,age,deceased,salary,Financiacion,Ahorro,Cuentas,diff_productos,compra,Antiguedad,mes
Index,,,,,,,,,,,,,,,,,,,,,
0,1375586,2018-01-28,2018-01-12,KHL,1.0,2,0,0,0,0,...,35,0,87218.10,0,0,1,0.0,0,1,2018-01
1,1050611,2018-01-28,2015-08-10,KHE,0.0,3,0,0,0,0,...,23,0,35548.74,0,0,1,0.0,0,30,2018-01
2,1050612,2018-01-28,2015-08-10,KHE,0.0,3,0,0,0,0,...,23,0,122179.11,0,0,1,0.0,0,30,2018-01
3,1050613,2018-01-28,2015-08-10,KHD,0.0,3,1,0,0,0,...,22,0,119775.54,0,1,0,0.0,0,30,2018-01
4,1050614,2018-01-28,2015-08-10,KHE,1.0,3,0,0,0,0,...,23,0,96796.23,0,0,1,0.0,0,30,2018-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5962919,1166765,2019-05-28,2016-08-14,KHE,0.0,3,0,0,0,0,...,22,0,43912.17,0,0,1,0.0,0,34,2019-05
5962920,1166764,2019-05-28,2016-08-14,KHE,0.0,3,0,0,0,0,...,23,0,23334.99,0,0,1,0.0,0,34,2019-05
5962921,1166763,2019-05-28,2016-08-14,KHE,1.0,2,0,0,0,0,...,47,0,96796.23,0,0,1,0.0,0,34,2019-05


In [25]:
df = df.drop(['entry_date','deceased','diff_productos'], axis=1)


In [26]:
df= pd.get_dummies(df, columns=['entry_channel'], drop_first=True)

In [27]:
#Generamos una copia para despues en el recomendador

df_rc2 = df.copy()

In [28]:
df = df.drop(['pk_cid','pk_partition'], axis = 1)

In [29]:
X = df.drop('compra', axis=1)  # Features
y = df['compra']  # Target

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [31]:
print("Tamaño del conjunto de entrenamiento (X):", X_train.shape)
print("Tamaño del conjunto de prueba (X):", X_test.shape)
print("Tamaño del conjunto de entrenamiento (y):", y_train.shape)
print("Tamaño del conjunto de prueba (y):", y_test.shape)

Tamaño del conjunto de entrenamiento (X): (4749306, 39)
Tamaño del conjunto de prueba (X): (1187327, 39)
Tamaño del conjunto de entrenamiento (y): (4749306,)
Tamaño del conjunto de prueba (y): (1187327,)


In [32]:
def AUC(model, X, y):
    return roc_auc_score(y, model.predict_proba(X)[:, 1])

def cross_validation(model, X, y):
    Kfold_metric = cross_val_score(model, X, y, cv=4, scoring='roc_auc')
    return Kfold_metric.mean()

In [33]:
label_ = []
metric_train_ = []
metric_valida_ = []

In [34]:
def evaluate_model(label, model, X, y):
 model.fit(X,y)
 metric_train = AUC(model,X,y)
 metric_valida = cross_validation(model,X,y)
 
 label_.append(label)
 metric_train_.append(metric_train)
 metric_valida_.append(metric_valida)
 return pd.DataFrame({'Model': label_,
                'AUC Train': metric_train_,
                'AUC Cross-Valida': metric_valida_
                }).sort_values('AUC Cross-Valida')

In [35]:
#evaluate_model(label='XGBoost', 
# model = xgb.XGBClassifier(), 
# X=X_train, y=y_train)

### Recomendador


In [36]:
df_rc=pd.read_csv("clusters_v2.csv",index_col=0)

In [37]:
df_rc2 = df_rc2[df_rc2["pk_partition"] == "2019-05-28"]

In [38]:
df_rc2

,pk_cid,pk_partition,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,...,entry_channel_KHD,entry_channel_KHE,entry_channel_KHF,entry_channel_KHK,entry_channel_KHL,entry_channel_KHM,entry_channel_KHN,entry_channel_KHO,entry_channel_KHQ,entry_channel_RED
Index,,,,,,,,,,,,,,,,,,,,,
5519930,657817,2019-05-28,0.0,3,0,0,0,0,0,0,...,False,True,False,False,False,False,False,False,False,False
5519932,657905,2019-05-28,1.0,1,0,0,0,0,0,1,...,False,False,False,False,False,False,False,False,False,False
5519933,657336,2019-05-28,1.0,2,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
5519934,657147,2019-05-28,0.0,2,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
5519935,658184,2019-05-28,1.0,2,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5962919,1166765,2019-05-28,0.0,3,0,0,0,0,0,0,...,False,True,False,False,False,False,False,False,False,False
5962920,1166764,2019-05-28,0.0,3,0,0,0,0,0,0,...,False,True,False,False,False,False,False,False,False,False
5962921,1166763,2019-05-28,1.0,2,0,0,0,0,0,0,...,False,True,False,False,False,False,False,False,False,False


In [39]:
df_rc2 = df_rc2[["pk_cid","salary","age","segment","gender","region_code"]]

In [40]:
df_rc3 = df_rc2.merge(df_rc["cluster"], left_on="pk_cid", right_index=True, how="inner")


In [41]:
df_rc3 = df_rc3.set_index("pk_cid")

In [42]:
df_cluster_5 = df_rc3[df_rc3["cluster"] == 5].copy()

In [45]:
df_cluster_5

,salary,age,segment,gender,region_code,cluster
pk_cid,,,,,,
657336,126765.57,38,2,1,28.0,5
655774,94054.20,37,2,0,15.0,5
656163,75408.45,53,2,0,28.0,5
656915,134470.14,37,2,1,33.0,5
656215,126765.57,38,2,1,28.0,5
...,...,...,...,...,...,...
1166804,99755.88,43,2,0,46.0,5
1166756,56105.28,42,2,0,50.0,5
1166734,96796.23,46,2,0,50.0,5


In [48]:
# eliminamos la columna de cluster del dataset para que no se use en el cálculo de la similitud de coseno.
df_cluster_5.drop("cluster", axis = 1, inplace = True)

KeyError: "['cluster'] not found in axis"

In [49]:
df_cluster_5 = df_cluster_5.sample(frac=0.25, random_state=42)

In [50]:
EXECUTE = True

In [52]:
if EXECUTE:
    # normalizamos nuestro dataset
    st = time.time()
    
    scaler = MinMaxScaler()
    df_normalized = scaler.fit_transform(df_cluster_5)
    df_normalized = pd.DataFrame(df_normalized, columns=df_cluster_5.columns)
    df_normalized.index = df_cluster_5.index


    et = time.time()
    print("Normalization took {} minutes".format(round((et - st)/60, 2)))

    sparse_ratings = sp.sparse.csr_matrix(df_normalized.values)

    st = time.time()
    print("Working with user similarity")
    user_similarity = cosine_similarity(sparse_ratings)
    user_sim_df = pd.DataFrame(user_similarity, index = df_normalized.index, columns = df_normalized.index)
    user_sim_df.columns = map(str, user_sim_df.columns)
    user_sim_df.to_parquet("cf_user_similarity_c.parquet.gzip")

    print("Working with item similarity")
    item_similarity = cosine_similarity(sparse_ratings.T)
    item_sim_df = pd.DataFrame(item_similarity, index = df_normalized.columns, columns = df_normalized.columns)
    item_sim_df.columns = map(str, item_sim_df.columns)
    item_sim_df.to_parquet("cf_item_similarity_c.parquet.gzip")

    et = time.time()
    print("Total time to calculate similarity took {} minutes.".format(round((et - st)/60, 2)))

else:
    user_sim_df = pd.read_parquet(os.path.join(PATH_FOLDER, "cf_user_similarity_c.parquet.gzip"))
    item_sim_df = pd.read_parquet(os.path.join(PATH_FOLDER, "cf_item_similarity_c.parquet.gzip"))

Normalization took 0.0 minutes
Working with user similarity
Working with item similarity
Total time to calculate similarity took 0.39 minutes.


In [53]:
user_sim_df.head(5)

,1382253,1013152,1122965,1528262,1201044,1212834,1397070,1264223,891992,1024508,...,1338900,1526945,1392407,1246381,1530117,1234413,1161091,1105276,1069446,1135038
pk_cid,,,,,,,,,,,,,,,,,,,,,
1382253,1.000000,0.781744,0.565610,0.953735,0.905412,0.861399,0.714250,0.903483,0.498013,0.718034,...,0.855158,0.956239,0.933672,0.891519,0.724265,0.629332,0.607654,0.643612,0.883815,0.942146
1013152,0.781744,1.000000,0.723801,0.631356,0.580087,0.571829,0.913830,0.504922,0.635962,0.917401,...,0.553201,0.632572,0.608547,0.593236,0.925591,0.804031,0.778030,0.823602,0.590270,0.563376
1122965,0.565610,0.723801,1.000000,0.444981,0.507029,0.448397,0.537533,0.537072,0.937769,0.678363,...,0.393991,0.420980,0.432344,0.590845,0.688628,0.803200,0.619858,0.816317,0.499866,0.487578
1528262,0.953735,0.631356,0.444981,1.000000,0.950285,0.916881,0.643553,0.938224,0.459069,0.673898,...,0.901232,0.999164,0.980082,0.958815,0.674500,0.622045,0.586798,0.601215,0.944586,0.978421
1201044,0.905412,0.580087,0.507029,0.950285,1.000000,0.773174,0.497285,0.961608,0.570496,0.632296,...,0.738996,0.946888,0.878284,0.929233,0.620676,0.609510,0.421169,0.516260,0.833652,0.971852


In [54]:
item_sim_df.head(5)

,salary,age,segment,gender,region_code
salary,1.000000,0.765749,0.850930,0.680040,0.795627
age,0.765749,1.000000,0.703912,0.615251,0.727477
segment,0.850930,0.703912,1.000000,0.702574,0.843236
gender,0.680040,0.615251,0.702574,1.000000,0.666934
region_code,0.795627,0.727477,0.843236,0.666934,1.000000


In [55]:
def top_users(user, df):
    '''
    This function prints the top 10 similar users based on cosine similarity.
    '''

    df.columns = map(int, df.columns)

    if user not in df.columns:
        return('No data available on user {}'.format(user))

    print('Most Similar Users:\n')

    sim_users = df.sort_values(by = user, ascending=False).index[1:10001]
    sim_values = df.sort_values(by = user, ascending=False).loc[:,user].tolist()[1:10001]

    for user, sim in zip(sim_users, sim_values):
        print('User #{0}, Similarity value: {1:.2f}'.format(user, sim))

    return sim_users, sim_values

In [56]:
user1 = np.random.choice(user_sim_df.index)


In [57]:
[similar_users, similar_values] = top_users(user1, user_sim_df)

Most Similar Users:

User #1531908, Similarity value: 1.00
User #591184, Similarity value: 1.00
User #1513890, Similarity value: 1.00
User #1537948, Similarity value: 1.00
User #1529422, Similarity value: 1.00
User #1469873, Similarity value: 1.00
User #1517483, Similarity value: 1.00
User #1534335, Similarity value: 1.00
User #1036011, Similarity value: 1.00
User #1014927, Similarity value: 1.00
User #1524954, Similarity value: 1.00
User #1108505, Similarity value: 1.00
User #1529866, Similarity value: 1.00
User #1506891, Similarity value: 1.00
User #754597, Similarity value: 1.00
User #1481638, Similarity value: 1.00
User #1178250, Similarity value: 1.00
User #1510095, Similarity value: 1.00
User #1263176, Similarity value: 1.00
User #1211940, Similarity value: 1.00
User #1454671, Similarity value: 1.00
User #1522233, Similarity value: 1.00
User #1526566, Similarity value: 1.00
User #1484880, Similarity value: 1.00
User #1328241, Similarity value: 1.00
User #1359512, Similarity value

In [58]:
similar_values

[1.0000000000000002,
 1.0000000000000002,
 1.0000000000000002,
 1.0000000000000002,
 1.0000000000000002,
 1.0000000000000002,
 1.0000000000000002,
 1.0000000000000002,
 1.0000000000000002,
 1.0000000000000002,
 1.0000000000000002,
 1.0000000000000002,
 1.0000000000000002,
 1.0000000000000002,
 1.0000000000000002,
 1.0000000000000002,
 1.0000000000000002,
 1.0000000000000002,
 1.0000000000000002,
 1.0000000000000002,
 1.0000000000000002,
 1.0000000000000002,
 1.0000000000000002,
 1.0000000000000002,
 1.0000000000000002,
 1.0000000000000002,
 1.0000000000000002,
 0.9999989072609264,
 0.999998749940747,
 0.9999961322971248,
 0.9999868272594906,
 0.9999508253572429,
 0.999943966694163,
 0.9998756633689148,
 0.9998705202404768,
 0.9997871122562982,
 0.9996283859988939,
 0.9996153010859006,
 0.9996006850455359,
 0.9996006850455359,
 0.9996006850455359,
 0.9996006850455359,
 0.9996006850455359,
 0.9996006850455359,
 0.9996006850455359,
 0.9996006850455359,
 0.9996006850455359,
 0.999600685045

In [59]:
media = np.mean(similar_values[:3300])
media

0.9641271214217619